In [3]:
import numpy as np
import torch
from torch import nn 
from torch.nn import Module

In [ ]:
class Convolutional_size_change(Module):
    def __init__(self, in_channels, kernel_size=3, stride=2,  downsample=True, padding=1):
        super().__init__()
        if downsample:
            self.conv = nn.Conv2d(in_channels=in_channels, out_channels=in_channels*2, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode="reflect")
            self.instance_norm = nn.InstanceNorm2d(in_channels*2)
        else:
            self.conv = nn.ConvTranspose2d(in_channels=in_channels, out_channels=in_channels/2, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode="reflect")
            self.instance_norm = nn.InstanceNorm2d(in_channels/2)
        self.activation = nn.ReLu(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.instance_norm(x)
        x = self.activation(x)

        return x
    



class ResidualBlock(nn.Module):
    def __init__(self, channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode="reflect")
        self.activation = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode="reflect")
        
    def forward(self, x):
        y = self.conv1(x)
        y = self.activation(y)
        y = self.conv2(y)
        return y + x
        


class Generator(nn.Module):
    def __init__(self, out_channels=1, in_channels=1, out_network_channels=64, number_of_residual_layers=9):

        self.first_conv = nn.Conv2d(in_channels=in_channels, out_channels=out_network_channels, kernel_size=7, stride=1, padding=3, padding_mode='reflect')
        self.first_norm = nn.InstanceNorm2d(out_network_channels)
        self.first_activ = nn.ReLU(inplace=True)
        self.down_first = Convolutional_size_change(out_network_channels)
        self.down_second = Convolutional_size_change(out_network_channels*2)

        self.residuals = []
        for i in range(number_of_residual_layers):
            residual = ResidualBlock(out_network_channels*4)
            self.residuals.append(residual)

        self.up_first = Convolutional_size_change(out_network_channels*4, downsample=False)
        self.up_second = Convolutional_size_change(out_network_channels*2, downsample=False)
        self.final_layer = nn.Conv2d(in_channels=out_network_channels, out_channels=out_channels, kernel_size=7, stride=1, padding=3, padding_mode='reflect')
